# Lesson08：The Humanoid Dream - G1 人形機器人

教學主題： 從四足到雙足，探索更進階的形態。

核心目標：理解人形機器人在控制上的複雜性與巨大潛力。

前 40 分鐘 (震撼登場)：

G1 展示：演示行走、爬樓梯、揮手，介紹設計與價格。

後 80 分鐘 (分組體驗)：

控制手臂：分組編程揮手、Give Me Five，討論四足與雙足差異。

## 8.0 Start up

站立开机步骤:

1. 挂起机器人；

2. “L2 + B” 进入阻尼模式；

3. “L2 + UP” 进入锁定站立模式；

4. 放下机器人，脚接触地面；

5. “R2 + A” 进入手柄控制模式；


## 8.1 Import and initialize the client for G1

In [ ]:
import time
import sys
from unitree_sdk2py.core.channel import ChannelSubscriber, ChannelFactoryInitialize
from unitree_sdk2py.go2.sport.sport_client import SportClient
from unitree_sdk2py.g1.loco.g1_loco_client import LocoClient

ChannelFactoryInitialize(0, "ens37")
sport_client = LocoClient()
sport_client.SetTimeout(10.0)
sport_client.Init()

## 8.2 Control the G1 to move

In [ ]:
# Move forward 0.3 m/s
sport_client.Move(0.3,0,0)

In [ ]:
# Move sideways 0.3 m/s
sport_client.Move(0,0.3,0)

In [ ]:
# Rotate at 0.3 rad/s
sport_client.Move(0,0,0.3)

## 8.3 Wave and shake hand like humans

In [ ]:
# shake hand twice with 3 seconds interval
sport_client.ShakeHand()

In [ ]:
# standup to squat
# need hung up and restart after doing this
sport_client.StandUp2Squat()

### Move a square with G1

In [ ]:
def move_continuous(v_x, v_y, duration, interval=0.4):
    """
    持续发送运动指令以保证机器人按指定速度运动指定时长
    
    Args:
        v_x (float): x方向速度
        v_y (float): y方向速度
        duration (float): 总运动时长（秒）
        interval (float): 发送指令的时间间隔（秒），小于单次指令运行时长0.5秒
    """
    start_time = time.time()
    while time.time() - start_time < duration:
        sport_client.Move(v_x, v_y, 0)
        time.sleep(interval)
    # 停止运动
    sport_client.Move(0, 0, 0)
    time.sleep(0.5)  # 确保停止指令生效

def move_square(base_vx=0.3, base_vy=0.3, scale=1.0, base_time=2.0):
    """
    控制机器人走正方形
    
    Args:
        scale (float): 控制正方形边长的比例因子，实际运动时间和速度会根据该因子调整
    """
    # 定义基础速度和基础时间，scale用于调整实际运动的时间（从而调整边长）
    base_vx = base_vx  # x方向基础速度（m/s）
    base_vy = base_vy  # y方向基础速度（m/s）
    base_time = base_time  # 基础运动时间（s），scale乘以该时间得到实际运动时间
    
    # 计算每条边的运动时长
    edge_duration = base_time * scale
    
    # 第一条边：x正方向运动
    move_continuous(base_vx, 0, edge_duration)
    
    # 第二条边：y正方向运动
    move_continuous(0, base_vy, edge_duration)
    
    # 第三条边：x负方向运动
    move_continuous(-base_vx, 0, edge_duration)
    
    # 第四条边：y负方向运动
    move_continuous(0, -base_vy, edge_duration)

if __name__ == "__main__":
    # 设置正方形大小的比例因子，scale越大，正方形边长越长
    square_scale = 1.0
    move_square(scale=square_scale)

### Move with arm

In [ ]:
from unitree_sdk2py.g1.arm.g1_arm_action_client import G1ArmActionClient
armAction_client = G1ArmActionClient()  
armAction_client.SetTimeout(10.0)
armAction_client.Init()

# Define action map
action_map = {
    "face wave": 25,
    "high wave": 26,
    "shake hand": 27,
}

In [ ]:
armAction_client.ExecuteAction(action_map.get("face wave"))

In [ ]:
armAction_client.ExecuteAction(25)

In [ ]:
armAction_client.ExecuteAction(action_map.get("high wave"))

In [ ]:
armAction_client.ExecuteAction(action_map.get("shake hand"))

## 8.4 Discussions about quadrupad and humanoid

四足机器人（如宇树 Go2）与双足人形机器人（如宇树 G1）在结构、控制逻辑、应用场景及技术难度等方面存在显著差异，同时也共享部分底层技术。以下从多个维度进行系统对比：



### 一、结构与运动能力

| 维度 | 四足机器人（Go2） | 双足机器人（G1） |
|------|------------------|------------------|
| 支撑点 | 4个支点，天然稳定 | 2个支点，动态平衡要求高 |
| 自由度 | 12个（每腿3个） | 23–43个（含手臂、腰、头等） |
| 运动模式 | 行走、小跑、跳跃、翻滚、上下楼梯（AI模式） | 行走、小跑（>2 m/s）、侧空翻、鲤鱼打挺、武术、跳舞、跨栏等 |
| 稳定性 | 静态/准静态稳定，抗干扰强 | 依赖实时力控与强化学习维持动态平衡 |
| 地形适应性 | 优异，尤其在非结构化地面（如草地、碎石） | 有限，需激光雷达+深度相机辅助，对地面平整度敏感 |

✅ 关键差异：四足机器人“稳”是天生优势；双足机器人“灵活拟人”是核心价值，但代价是控制复杂度指数级上升。

### 二、感知与智能

Go2：

* 搭载自研 4D 激光雷达 L1（360°×90°，最小探测距离 0.05m）

* 前置摄像头 + 麦克风 + 扬声器

* 支持基础语音交互、手势识别、建图导航

* AI 模式下可执行预设高难度动作，但自主决策能力弱

G1：

* 头部集成 Livox MID-360 激光雷达 + Intel RealSense D435i 深度相机

* 支持 UnifoLM 大模型（宇树统一大模型），具备云交互能力

* 可理解复杂指令（如“跳个舞”“比个心”），但语义理解仍有限（如无法回答“展会后去哪吃饭”）

* 动作通过 深度强化学习 + 仿真训练 自动生成

🔍 共性：两者都依赖多传感器融合与AI算法，但 G1 的“具身智能”探索更深，Go2 更偏向“遥控+有限自主”。

### 三、应用场景

| 场景 | Go2 更适合 | G1 更适合 |
|------|------------|-----------|
| 家庭陪伴 | ✔️ 体积小、安全性高、社交属性强（遛狗引围观） | ❌ 体型大、速度快、有碰撞风险 |
| 教育开发 | ✔️ EDU 版开源 ROS2，支持 SLAM、动作编程（中小学到高校） | ✔️ G1 Edu 版支持二次开发，用于人形运控研究 |
| 工业巡检 | ✔️ 可穿越狭窄空间，搭载传感器做变电站监测 | ✔️ 单臂负载 1.5kg，可在工厂辅助装配 |
| 特种任务 | ✔️ 防护等级高，可改装为消防搜救、化工巡检 | ⚠️ 负重更大但续航短（仅2小时），部署受限 |
| 娱乐表演 | ✔️ 才艺展示（握手、扑人） | ✔️ 舞蹈、格斗、演唱会伴舞（已登王力宏舞台） |

💡 趋势：Go2 是“工具型智能体”，G1 是“拟人型智能体”——前者重功能落地，后者重形态与交互突破。

### 四、技术演进关系

正如行业共识所言：
“四足是双足的初级阶段。”

宇树自身发展路径印证此逻辑：

* 2013 年 XBot → 2023 年 Go2（成熟四足平台）

* 2024 年推出 G1（基于四足积累的电机、减速器、运控算法迁移到人形）

共享技术栈包括：
* 高扭矩关节电机

* 实时运动规划算法

* 激光雷达 SLAM

* 强化学习框架


但双足需额外攻克：
* 全身动力学建模

* ZMP（零力矩点）平衡控制

* 双臂协同操作

* 人机自然交互（眼神、姿态、语言）


### 五、现实局限与未来展望

Go2 的短板：

软件生态不成熟，多数功能停留在演示阶段，缺乏主动服务意识。

G1 的挑战：

续航仅 2 小时，价格高达 9.9 万元，且自主决策仍处 L2 级（需人工干预突发状况）。

共同瓶颈：

具身智能尚未突破——机器人能“动”，但不能真正“理解环境并主动解决问题”。

🌟 展望：随着 UnifoLM 等大模型与机器人本体深度融合，未来 2–3 年或迎来“感知-决策-执行”闭环的质变。届时，Go2 可能成为家庭管家，G1 或化身导览员、护理助手。


### 总结

| 对比项 | 四足（Go2） | 双足（G1） |
|--------|-------------|------------|
| 定位 | 仿生工具 / 入门平台 | 拟人智能体 / 技术旗舰 |
| 优势 | 稳定、灵活、性价比高 | 拟人、交互强、扩展性好 |
| 适合人群 | 科技爱好者、教育机构、轻工业用户 | 高校实验室、开发者、前沿企业 |
| 发展阶段 | 产品化初期（硬件成熟，软件待完善） | 技术验证期（运动强，智能弱） |

简言之：Go2 是“会动的狗”，G1 是“学走路的人”——一个重实用，一个重未来。选择取决于你是想“用”，还是想“参与进化”。